In [2]:
# ! pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental

In [11]:
## all tools are runnables as they contain 'invoke' function
# Built-in Tool - DuckDuckGo Search
from langchain_community.tools import DuckDuckGoSearchRun

In [5]:
search_tool = DuckDuckGoSearchRun()
result = search_tool.invoke("What is the latest news of today?")
print(result)

NPR news, audio, and podcasts. Coverage of breaking stories, national and world news, politics, business, science, technology, and extended coverage of major national and world events. Live News journalism from around the world. Prof Iain Gillespie, former principal and vice-chancellor of the University of Dundee, is accused of creating a financial crisis and walking away. Reuters.com is your online source for the latest world news stories and current events, ensuring our readers up to date with any breaking news developments Latest U.S. Video. 47:15. ABC News Live Prime: June 24, 2025. Jun 24, 2025. 4:55. ... joins ABC News' Linsey Davis to detail how Iranian civilians have been impacted by Israeli strikes. Stay on top of News latest developments on the ground with Al Jazeera's fact-based news, exclusive video footage, photos and updated maps.


In [6]:
result = search_tool.invoke("What is the status of Iran and Iraq war as of today?")
print(result)

The Iran-Israel ceasefire that ended 12 days of conflict appears to still be holding. The Pentagon held a widely anticipated briefing giving new details on the US strikes against Iran's nuclear ... Iran's military said it targeted America's Al Udeid Air Base in Qatar with a "devastating and powerful" missile attack today in retaliation for the U.S. attack on three of Iran's nuclear ... The Iran-Israel cease-fire appears to hold in the Middle East. President Trump pushed back on an intelligence report saying the strikes only set back Tehran's nuclear ambitions by a few months. "The War Powers Resolution we've introduced today orders the removal of U.S. Armed Forces from hostilities against Iran, while allowing U.S. forces to carry out defensive operations to defend ... Trump prevailed after mocking his rival as "low energy" and railing against his brother President George W Bush's war in Iraq. ... in Iran. Meanwhile, Iranian state television shows scenes of ...


In [7]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


In [9]:
# Built-in Tool - Shell Tool
from langchain_community.tools import ShellTool

shell_search_tool = ShellTool()

In [10]:
results = shell_search_tool.invoke('ls')
print(results)

c:\Shivangi\My_Learning\2_LangChain\langchainEnv\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


Executing command:
 ls
'ls' is not recognized as an internal or external command,
operable program or batch file.



In [ ]:
## Custom tools -
# Method 1 - By using @tool decorator
from langchain_core.tools import tool

# Step 1 - Create a function
def multiplication(a,b):
    """multiplication of two numbers"""
    return a*b

In [5]:
# Step 2 - Add type hints
def multiplication(a:int, b:int) -> int:
    """multiplication of two numbers"""
    return a*b

In [6]:
# Step 3 - Add tool decorator
@tool
def multiplication(a:int, b:int) -> int:
    """multiplication of two numbers"""
    return a*b

In [9]:
result = multiplication.invoke({'a':4, 'b':5})

In [10]:
print(result)

20


In [12]:
print(multiplication.name) ## tool name
print(multiplication.description) ## description that we gave
print(multiplication.args) ## arguments (parameters) that we gave

multiplication
multiplication of two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [13]:
print(multiplication.args_schema.model_json_schema())

{'description': 'multiplication of two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiplication', 'type': 'object'}


In [14]:
# Method 2 - By using StructuredTool along with pydantic
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [15]:
class MultiplyInput(BaseModel):
    a:int = Field(required = True, description="The first number to add")
    b:int = Field(required = True, description="The second number to multiply")

In [ ]:
def multiplication(a:int,b:int) -> int:
    return a*b

In [17]:
multiplication_tool = StructuredTool.from_function(
    func=multiplication,
    name= "let's multiply",
    description="multiplication of two numbers",
    args_schema=MultiplyInput
)

In [18]:
result = multiplication_tool.invoke({'a':7, 'b':5})
print(result)

35


In [19]:
print(multiplication_tool.name)
print(multiplication_tool.description)
print(multiplication_tool.args)

let's multiply
multiplication of two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to multiply', 'required': True, 'title': 'B', 'type': 'integer'}}


In [20]:
# Method 3 - By using BaseTool class
from langchain.tools import BaseTool
from typing import Type

In [22]:
## arg_schema using pydantic
class AdditionInput(BaseModel):
    a:int = Field(required=True, description='The first number used to perform addition')
    b:int = Field(required=True, description='The second number used to perform the addition')

In [23]:
class AdditionTool(BaseTool):
    name:str = "let's add"
    description:str = 'addition of two numbers'
    args_schema:Type[BaseModel] = AdditionInput

    def _run(self, a:int, b:int) -> int:
        return a+b

In [26]:
addition_tool = AdditionTool()
result = addition_tool.invoke({'a':10, 'b':1})

In [28]:
print(result)

11


In [29]:
print(addition_tool.name)
print(addition_tool.description)
print(addition_tool.args)

let's add
addition of two numbers
{'a': {'description': 'The first number used to perform addition', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number used to perform the addition', 'required': True, 'title': 'B', 'type': 'integer'}}


In [30]:
# Toolkit
# Custom tools
## we will have to create two tools in order to create Toolkit
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [31]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]

In [32]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)

add => Add two numbers
multiply => Multiply two numbers
